In [1]:
import numpy as np
from collections import defaultdict

import scipy.sparse as sparse
import scipy.sparse.linalg as sparse_linalg
from scipy.spatial import cKDTree

np.set_printoptions(suppress=True)

In [2]:
'''
description：read obj file
input: obj file name
return: obj vertices and face indices
'''
def readOBJ(filename):
    vertices = []
    vertex_norm = []
    vertex_tex = []
    triangles = []
    texcoords = []
    for line in open(filename, "r"):
        values = line.split()
        if(values==[]):
            continue
        if(values=='#'):
            continue
        if(values[0]=='v'):
            vertices.append([float(values[1]),float(values[2]),float(values[3])])
        if(values[0]=='vn'):
            vertex_norm.append([float(values[1]),float(values[2]),float(values[3])])
        if(values[0]=='vt'):
            vertex_tex.append([float(values[1]),float(values[2]),float(values[3])])
        if(values[0]=='f'):
            face=[]
            texcoord = []
            norm = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if(len(w)>=2 and len(w[1])>0):
                    texcoord.append(int(w[1]))
                else:
                    texcoord.append(-1)
                if(len(w)>=3 and len(w[2])>0):
                    norm.append(int(w[2]))
                else:
                    norm.append(-1)
            triangles.append(face)
            texcoords.append(texcoord)
    return np.array(vertices),np.array(triangles)-1

In [3]:
'''
计算邻接三角面
'''
def compute_adjacent_by_edges(objFaces):
    # 每条边涉及到哪些面
    candidates = defaultdict(set)
    for i in range(objFaces.shape[0]):
        f0, f1, f2 = sorted(objFaces[i])
        candidates[(f0, f1)].add(i) # 注意i是面索引
        candidates[(f0, f2)].add(i)
        candidates[(f1, f2)].add(i)
    # 每个面与哪些面邻接；candidates的value存的就是共享边的面
    faces_adjacent = defaultdict(set)  # Face -> Faces
    for faces in candidates.values():
        for f in faces:
            faces_adjacent[f].update(faces)
    # 按面的顺序排列所有邻接面
    faces_sort = []
    for f, adj in faces_adjacent.items():
        exclude_f = []
        for a in adj :
            if a != f:
                exclude_f.append(a)
        faces_sort.append([f,exclude_f])
    faces_sort = sorted(faces_sort, key=lambda e: e[0])
    # 只返回邻接面
    faces_adj = []
    for _,ff in faces_sort:
        faces_adj.append(ff)
    return faces_adj

In [4]:
'''
按照论文计算面法线，并将法线作为每个面的第4个顶点
返回：
第1维代表面索引，第2、3维记录了三个方向，每列为一个方向(面2个方向，法线1个方向)
新的顶点：在原始基础上添加了法线顶点
新的面索引：法线顶点的索引
'''
def compute_face_norms(verts,faces):
    fnorms = []
    v4 = []
    # 计算每个面片的三组方向
    for f in faces:
        v1 = verts[f[0]]
        v2 = verts[f[1]]
        v3 = verts[f[2]]
        a = v2 - v1
        b = v3 - v1
        tmp = np.cross(a, b)
        c = tmp.T / np.linalg.norm(tmp)
        fnorms.append([a,b,c])
        # 更新顶点，添加第四个顶点
        v4.append(v1+c)
    fnorms = np.array(fnorms)     
    v4 = np.array(v4)
    new_verts = np.concatenate((verts,v4),axis=0)
    # 更新面片顶点索引
    v4_indices = np.arange(verts.shape[0],verts.shape[0]+v4.shape[0])
    new_faces = np.concatenate((faces,v4_indices.reshape((-1,1))),axis=1)
    return np.transpose(fnorms, (0, 2, 1)),new_verts,new_faces

'''
计算顶点法线：面法线的平均
'''
def compute_vert_norms(faceNorms,faces):
    vf = defaultdict(set)
    # 每个顶点与哪些面片有关
    for n, (f0, f1, f2) in enumerate(faces[:, :3]):
        vf[f0].add(n)
        vf[f1].add(n)
        vf[f2].add(n)
    # 计算每个顶点的法线，用面法线的平均
    vn = np.zeros((len(vf),3))
    for i in range(len(vf)):
        vn[i] = np.mean(faceNorms[list(vf[i])],axis=0)
        vn[i] = vn[i] / np.linalg.norm(vn[i])
    return vn

In [5]:
'''
创建稀疏矩阵
'''
row = np.array([0, 1, 2] * 4)
def expand(f, inv, size):
    i0, i1, i2, i3 = f
    col = np.array([i0, i0, i0, i1, i1, i1, i2, i2, i2, i3, i3, i3])
    data = np.concatenate([-inv.sum(axis=0), *inv])
    return sparse.coo_matrix((data, (row, col)), shape=(3, size), dtype=float)
def construct(faces, invVs, size):
        assert len(faces) == len(invVs)
        return sparse.vstack([expand(f, inv, size) for f, inv in zip(faces, invVs)], dtype=float)

In [6]:
'''
定义损失函数：identity_cost
'''
def construct_identity_cost(faces,invVs,verts):
    AEi = construct(faces,invVs,verts.shape[0])
    AEi.eliminate_zeros()
    Bi = np.tile(np.identity(3, dtype=float), (len(faces), 1))
    assert AEi.shape[0] == Bi.shape[0]
    return AEi.tocsr(), Bi

'''
定义损失函数:smooth_cost
'''
def construct_smoothness_cost(faces,invVs,verts,adjacent):
    # 预先计算并保存
    lhs = []
    rhs = []
    face_idx = 0
    for f,inv in zip(faces,invVs):
        for adjIndex in adjacent[face_idx]:            
            lhs.append(expand(f,inv,verts.shape[0]).tocsc())
            rhs.append(expand(faces[adjIndex],invVs[adjIndex],verts.shape[0]).tocsc())
        face_idx = face_idx + 1
    AEs = sparse.vstack(lhs) - sparse.vstack(rhs)
    sparse.save_npz("cat-lion.npz",AEs)
#     AEs = sparse.load_npz("head.npz")
    AEs.eliminate_zeros()
    
    count_adjacent = sum(len(a) for a in adjacent)
    Bs = np.zeros((count_adjacent * 3, 3))
    assert AEs.shape[0] == Bs.shape[0]
    return AEs, Bs

In [7]:
'''
解方程 `Ax=b`
'''
def apply_markers(A, b, target, markers):
    """
    Ei的形变量接近单位阵
    Es的形变量是邻接面的形变量接近
    :param A: Matrix (NxM)
    :param b: Result vector (Nx3)
    :param target: Target mesh
    :param markers: Marker (Qx2) with first column the source indices and the second the target indices.
    :return: Matrix (Nx(M-Q)), result vector (Nx3)
    """
    assert markers.ndim == 2 and markers.shape[1] == 2
    invmarker = np.setdiff1d(np.arange(A.shape[1]), markers[:, 0])# 不在marker中的点
    zb = b - A[:, markers.T[0]] * target[markers.T[1]] # 使得形变量接近b
    return A[:, invmarker].tocsc(), zb

In [8]:
def get_closet_points(kd_tree,verts,verts_norms,target_normal,max_angle=np.radians(90),ks=200):
    '''
    寻找最近点，同时满足法线角度差小于max_angle,寻找最近点的数量不大于ks
    '''
    assert len(verts) == len(verts_norms),"source verts and norms is not same length"
    closest_points = []
    dists,indices = kd_tree.query(verts,min(len(target_normal),ks)) # 找到在最近点的距离和索引
    for i,(dist,ind) in enumerate(zip(dists,indices)):
        angles = np.arccos(np.dot(target_normal[ind],verts_norms[i]))
        if((np.abs(angles)<max_angle).any()):
            cind = ind[np.abs(angles)<max_angle][0]
            closest_points.append([i,cind])
    return np.array(closest_points)

In [9]:
def revert_marks(A,x,target_v,markers,out):
    if out is None:
        out = np.zeros((A.shape[1] + len(markers), 3))
    else:
        assert out.shape == (A.shape[1] + len(markers), 3)
    invmarker = np.setdiff1d(np.arange(out.shape[0]),markers[:,0])
    out[invmarker] = x
    out[markers[:,0]] = target_v[markers[:,1]]
    return out

In [10]:
# 每个面片的中心
def get_centroids(v,f):
    return v[f[:,:3]].mean(axis=1)

def max_triangle_length(v,faces):
    fnorms = []
    max_len = 0.0
    # 计算每个面片的三组方向
    for f in faces:
        v1 = v[f[0]]
        v2 = v[f[1]]
        v3 = v[f[2]]
        a = v2 - v1
        b = v3 - v1
        if(max_len<max(max_len,np.linalg.norm(a),np.linalg.norm(b))):
            max_len = max(max_len,np.linalg.norm(a),np.linalg.norm(b))
    return max_len

def get_closet_triangles(vn1,vn2,center1,center2,radius):
    max_angle = np.radians(90)
    k = 500
    triangles = set()
    kkd_tree = cKDTree(center2)
    dists,indicies = kkd_tree.query(center1,min(center2.shape[0],k))
    for index_source,(dist,ind) in enumerate(zip(dists,indicies)):
        angles = np.arccos(np.dot(vn2[ind],vn1[index_source]))
        if((angles<max_angle).any()):
            index_target = ind[angles<max_angle][0]
            triangles.add((index_source,index_target))
    return triangles

def match_triangles(v1,v2,f1,f2,vn1,vn2,factor=2):
    source_f_center = get_centroids(v1,f1)
    target_f_center = get_centroids(v2,f2)   
    radius = max(max_triangle_length(v1,f1),max_triangle_length(v2,f2))*factor 
    triangles = get_closet_triangles(vn1,vn2,source_f_center,target_f_center,radius)
    tmp_triangles = get_closet_triangles(vn2,vn1,target_f_center,source_f_center,radius)
    triangles.update((t[1],t[0]) for t in tmp_triangles)
    return list(triangles)

# 逐步进行

In [11]:
# 手动挑选的一部分对应点
# marks=np.array([[14387,14537],[17495,1775], [17661,1773], [17514,7180],[17518,7223], [23208,5808], [22815,14658], [23380,9424], [22954,4674], 
# [22807,14441],[22945,5914], [22825,15615], [22963,15629], [4694,13863],  [10481,13664], [5868,1998],  [5979,1957], [6173,7448], [6332,7406],    
# [5182,1912], [6834,1929], [5500,7361], [7172,14939], [3240,1496], [3291,11681], [28675,14405], [14308,6597], [14619,9671], [1078,6203],
# [779,11588],[24739,2486], [25242,7923],[11,4241]])
marks = np.array([[2653,1316],[2620,1412],[2141,920],[1570,827],[6855,4585 ],[6533,4655],[5752,4199 ],[5819,4130 ],[5185,3998 ],[6194,4454 ],
                  [6313,4456 ],[5257,3985 ],[2422,1156],[1441,670],[2405,1224],[1430,738],[648,263],[173,66],[82,40],[7179,4979],[5106,3913],
                  [288,4365],[190,84],[444,253],[1151,345],[786,489],[38,3904],[266,90],[912,394],[3272,1603],[4967,2285],[3947,3623],
                  [3401,2775],[4988,2654],[3999,3682],[4315,1859],[3454,2875],[4263,1811],[3510,2889],[4078,1833],[4213,2727],[4186,2733],
                  [4181,2822],[4010,3778],[4047,3762],[3587,3839],[3540,2804],[3677,2581],[4698,2422],[3804,3056],[4816,2398],[4773,2321],
                  [3757,3453],[3695,3268],[3236,1641]])

In [12]:
# 读取原始模型顶点和面片
source_org_v,source_org_f = readOBJ("./model/cat-lion/cat/cat-reference.obj")#readOBJ("./model/face/sourceface/face-reference.obj")#
target_org_v,target_org_f = readOBJ("./model/cat-lion/lion/lion-reference.obj") #readOBJ("./model/face/targetface/head-reference.obj")#

In [13]:
# 计算邻接面
source_face_adj = compute_adjacent_by_edges(source_org_f)
target_face_adj = compute_adjacent_by_edges(target_org_f)

In [14]:
# 计算法线，重置顶点和面片索引(因为对每个面添加第四个顶点)
source_f_norms,source_v,source_f = compute_face_norms(source_org_v,source_org_f)
invVs = np.linalg.inv(source_f_norms)
target_f_norms,target_v,target_f = compute_face_norms(target_org_v,target_org_f)

In [15]:
# 计算Ei损失
Ei,Bi = construct_identity_cost(source_f,invVs,source_v)
EiSelect,EiLoss = apply_markers(Ei,Bi,target_v,marks)
Es,Bs = construct_smoothness_cost(source_f,invVs,source_v,source_face_adj)
EsSelect,EsLoss = apply_markers(Es,Bs,target_v,marks)

In [16]:
# 建立KD树，便于查找最近点
kd_tree_target = cKDTree(target_org_v)
target_vn = compute_vert_norms(target_f_norms[...,-1],target_org_f)

In [17]:
# 论文中的迭代
# 损失函数权重
Ws = 1.0
Wi = 0.001
Wc = [0, 10, 50, 250, 1000, 2000, 3000, 5000]

iterations = len(Wc)
total_steps = 3  # Steps per iteration

In [18]:
vertices_copy = source_v.copy()
vertices_fnorm_copy = source_f_norms.copy()
for iteration in range(iterations):
    Astack = [EiSelect * Wi, EsSelect * Ws]
    Bstack = [EiLoss * Wi, EsLoss * Ws]    
    if(iteration>0 and Wc[iteration]!=0):
        AEc = sparse.identity(source_v.shape[0], dtype=float, format="csc")[:source_org_v.shape[0]]
        vertices_clipped = vertices_copy[:source_org_v.shape[0]]        
        # 从source中获取到与target顶点的最近点，且满足法线夹角约束
        closest_points = get_closet_points(kd_tree_target,vertices_clipped,compute_vert_norms(vertices_fnorm_copy[...,-1],source_org_f),target_vn)
        AEc = AEc[closest_points[:,0]]
        Bc = target_v[closest_points[:,1]]        
        mAEc,mBc = apply_markers(AEc,Bc,target_v,marks)
        Astack.append(mAEc * Wc[iteration])
        Bstack.append(mBc * Wc[iteration])        
        
    A = sparse.vstack(Astack,format="csc")
    A.eliminate_zeros()
    b = np.concatenate(Bstack)
    LU = sparse_linalg.splu((A.T @ A).tocsc())
    x = LU.solve(A.T @ b)
    # 重建顶点
    vertices_copy = revert_marks(A,x,target_v,marks,vertices_copy) 
    vertices_fnorm_copy,vertices_copy,_ = compute_face_norms(vertices_copy[:source_org_v.shape[0]],source_org_f)

In [19]:
result_v = vertices_copy
result_f = source_org_f
result_fn = vertices_fnorm_copy[...,-1]

In [20]:
coresponding = match_triangles(result_v,target_v,result_f,target_f,result_fn,target_f_norms[...,-1])

In [21]:
import pickle
with open("cat-lion.txt", "wb") as fp:   #Pickling
    pickle.dump(coresponding, fp)